In [1]:
!pip install snakeviz

In [2]:
from kafka import KafkaProducer
import os
import io
import logging
import psutil
import time
import cProfile
import pstats


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


topic = os.environ.get('KAFKA_TOPIC', "cpu-logger")
logger.info(f"Kafka topic: {topic}")

_producer = None
try:
    _producer = KafkaProducer(bootstrap_servers=os.environ.get('KAFKA_BROKER', 'broker1:9093').split(","), 
                              api_version=(0, 10),
                              max_block_ms=10000)
except Exception as ex:
    logger.error('Exception while connecting Kafka', exc_info=True)


num_runs = 1
run = 0
loops = 100

while run < num_runs:
    profiler = cProfile.Profile()
    profiler.enable()

    for i in range(loops):
        try:
            message = ','.join([str(p) for p in psutil.cpu_percent(interval=0, percpu=True)])
            value_bytes = bytes(message, encoding='utf-8')
            _producer.send(topic=topic, 
                           value=value_bytes)
        
        except Exception as ex:
            logger.error('Exception in publishing message', exc_info=True)
    _producer.flush()
    profiler.disable()

    if run % 10 == 0:
        print(f"Run {run} done!")
    run += 1

    time.sleep(0.5)

print("Done with logging")

INFO:__main__:Kafka topic: cpu-logger
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=broker1:9093 <connecting> [IPv4 ('172.18.0.6', 9093)]>: connecting to broker1:9093 [('172.18.0.6', 9093) IPv4]
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=broker1:9093 <connecting> [IPv4 ('172.18.0.6', 9093)]>: Connection complete.
INFO:kafka.conn:<BrokerConnection node_id=1 host=broker1:9093 <connecting> [IPv4 ('172.18.0.6', 9093)]>: connecting to broker1:9093 [('172.18.0.6', 9093) IPv4]
INFO:kafka.conn:<BrokerConnection node_id=1 host=broker1:9093 <connecting> [IPv4 ('172.18.0.6', 9093)]>: Connection complete.
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=broker1:9093 <connected> [IPv4 ('172.18.0.6', 9093)]>: Closing connection. 


Run 0 done!
Done with logging


In [3]:
profiler.dump_stats("cpu-logger.prof")

In [ ]:
# the cell magic does not work, no clue why
# %load_ext snakeviz
# %snakeviz "test.prof"
# so use the command line and the local browser to view the results
import os
print("Open the URL: http://127.0.0.1:9001/snakeviz/%2Fhome%2Fjovyan%2Fcpu-logger.prof in your local browser. Interrupt the cell to stop the server.")
os.system('snakeviz "cpu-logger.prof" -s -p 9001 -H 0.0.0.0')

Open the URL: http://127.0.0.1:9001/snakeviz/%2Fhome%2Fjovyan%2Fcpu-logger.prof in your local browser. Interrupt the cell to stop the server.


404 GET /static/vendor/lodash.min.js.map (172.18.0.1) 2.41ms
